In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_aac_animals import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "snhu1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# vars for grazioso logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    # display grazioso logo
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode('utf-8')),
             style={'display': 'block','margin':'auto','textAlign':'center', 'width':'10%'}),
    # display unique identifier
    html.Div("Developed By: Allen Jhane Dela Cruz", style={'textAlign': 'center','color':'#FB8DA0', 'fontSize':20}), 
    html.Br(),
    html.Hr(),
    
    # display filter options
    html.Div("Filters", style={'fontSize':18, 'fontWeight': 'old'}),
    # filtering options as radio items
    dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',  # Default value
            labelStyle={'display': 'block'}  # Display options vertically
        ),
    
    # display table
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         
        # features for your interactive data table to make it user-friendly for your client
        row_selectable = "single", # adds selection radio button
        page_size = 10,  # displays 20 to a page
        style_cell = {'textAlign': 'left'}, # aligns all cells to the left
        selected_rows = [0], # default selected row is id = 0
        selected_columns = [], # so default selected column is not NULL
        sort_action = "native", # adds a sorting mechanism for the user
        filter_action = "native", # allows user to filter by typing a word
        # hides columns user won't care about so table will fit on page without scrolling to the right
        hidden_columns = ['location_lat', 'location_long', 'rec_num'],
        
        # every other row is gray for better readability
        style_data_conditional=[{
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(220, 220, 220)',}],
        
        # header is blue and bold
        style_header={
        'backgroundColor': 'rgb(186, 255, 255)',
        'color': 'black',
        'fontWeight': 'bold',
        'paddingRight': 20,
        },
        
        # hides toggle button for hidden columns
        css=[{"selector": ".show-hide", "rule": "display: none"}],
    
    ),
    
    html.Br(),
     html.Hr(),
    
    # display map and graph in-line to see both together
    html.Div([
             html.Div(id='map-id', style={'width': '50%', 'display': 'inline-block'}),
            html.Div(id='graph-id', style={'width': '50%', 'display': 'inline-block'})
    ], style={'margin': 'auto', 'width': '100%'})

])

#############################################
# Interaction Between Components / Controller
#############################################
# get an input of filter-type when a filter is selected
# output the data and columns that will be updated within the datatable
@app.callback(Output('datatable-id','data'),
              Output('datatable-id','columns'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # if a certain filter is selected, query based on selection 
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", 
                              "Chesa Bay Retr Mix", 
                              "Newfoundland Mix",
                              "Newfoundland/Labrador Retriever"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))                               
    
    elif filter_type == 'mountain':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ['German Shepherd', 
                              'German Shepherd Mix',
                              'German Shepherd/Alaskan Malamute', 
                              'German Shepherd/Rottweiler',
                              'German Shepherd/Siberian Husky',
                              'Alaskan Malamute', 
                              'Old English Sheepdog', 
                              'Siberian Husky', 
                              'Siberian Husky Mix',
                              'Siberian Husky/German Shepherd',
                              'Rottweiler',
                              'Rottweiler Mix', 
                              'Rottweiler/German Shepherd',
                              'Rottweiler/Siberian Husky']},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
        
    elif filter_type == 'disaster': 
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ['Doberman Pinsch', 
                              'German Shepherd', 
                              'Golden Retriever Mix', 
                              'Bloodhound Mix', 
                              'Bloodhound', 
                              'Rottweiler', 
                              'Rottweiler Mix', 
                              'Rottweiler/German Shepherd']},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        })) 
        
    else:
        df = pd.DataFrame.from_records(db.read({})) # Reset option, no filter applied
    
    # remove object _id from list since it is not a necessary item and will cause error
    df.drop(columns=['_id'],inplace=True)
    
    # update columns and data for the datatable based on the query
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return [data, columns]

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
            return []  # Return an empty list if data is None or empty

    df = pd.DataFrame(viewData)

    # Count the occurrences of each breed
    breed_counts = df['breed'].value_counts()

    # return a piechart with breeds and values
    return [
        dcc.Graph(
        figure=px.pie(names=breed_counts.index, 
                      values=breed_counts.values, 
                      title='Preferred Animals'),
        style={'width': '100%', 'height': '500px'}
    )
    ]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
           center=[30.75,-97.48], zoom=6, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for the map
           # displays a popup message at marker with type, breed, color, age, and location of selected animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([ 
                html.H1(dff.iloc[row,9]),
                html.P("Type: " + dff.iloc[row,3]),
                html.P("Breed: " + dff.iloc[row,4]),
                html.P("Color: " + dff.iloc[row,5]),
                html.P("Age: " + dff.iloc[row,1]),
                html.P("Location: " + "{}, {}".format(dff.iloc[row,13], dff.iloc[row,14]))
             ])
          ])
       ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:28031/
